Исходный датасет: https://www.kaggle.com/jsphyg/weather-dataset-rattle-package

In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler 

import warnings

from tqdm import tqdm

from pandas.plotting import scatter_matrix


warnings.filterwarnings("ignore")

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (18,8)



Удалим столбец RISK_MM, т.к. он непосредственно влияет на целевую переменную RainTomorrow.

In [16]:
HOUSING_PATH = "data/" 
def load_weather_data(housing_path=HOUSING_PATH):    
    csv_path = os.path.join(housing_path, "weatherAUS.csv")   
    return pd.read_csv(csv_path).drop("RISK_MM", axis = 1)

In [17]:
weather_df = load_weather_data()

print("Размер датасета:", weather_df.shape)

weather_df.head()

Размер датасета: (142193, 23)


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [21]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   Location       142193 non-null  object 
 2   MinTemp        141556 non-null  float64
 3   MaxTemp        141871 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   Evaporation    81350 non-null   float64
 6   Sunshine       74377 non-null   float64
 7   WindGustDir    132863 non-null  object 
 8   WindGustSpeed  132923 non-null  float64
 9   WindDir9am     132180 non-null  object 
 10  WindDir3pm     138415 non-null  object 
 11  WindSpeed9am   140845 non-null  float64
 12  WindSpeed3pm   139563 non-null  float64
 13  Humidity9am    140419 non-null  float64
 14  Humidity3pm    138583 non-null  float64
 15  Pressure9am    128179 non-null  float64
 16  Pressure3pm    128212 non-null  float64
 17  Cloud9am       88536 non-null

In [23]:
weather_df.describe(include="all")

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
count,142193,142193,141556.000000,141871.000000,140787.000000,81350.000000,74377.000000,132863,132923.000000,132180,...,140419.000000,138583.000000,128179.000000,128212.000000,88536.000000,85099.000000,141289.000000,139467.000000,142193,142193
unique,3436,49,NaN,NaN,NaN,NaN,NaN,16,NaN,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2
top,2016-07-14,Canberra,NaN,NaN,NaN,NaN,NaN,W,NaN,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,No
freq,49,3418,NaN,NaN,NaN,NaN,NaN,9780,NaN,11393,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110313,110316
mean,NaN,NaN,12.186400,23.226784,2.349974,5.469824,7.624853,NaN,39.984292,NaN,...,68.843810,51.482606,1017.653758,1015.258204,4.437189,4.503167,16.987509,21.687235,NaN,NaN
std,NaN,NaN,6.403283,7.117618,8.465173,4.188537,3.781525,NaN,13.588801,NaN,...,19.051293,20.797772,7.105476,7.036677,2.887016,2.720633,6.492838,6.937594,NaN,NaN
min,NaN,NaN,-8.500000,-4.800000,0.000000,0.000000,0.000000,NaN,6.000000,NaN,...,0.000000,0.000000,980.500000,977.100000,0.000000,0.000000,-7.200000,-5.400000,NaN,NaN
25%,NaN,NaN,7.600000,17.900000,0.000000,2.600000,4.900000,NaN,31.000000,NaN,...,57.000000,37.000000,1012.900000,1010.400000,1.000000,2.000000,12.300000,16.600000,NaN,NaN
50%,NaN,NaN,12.000000,22.600000,0.000000,4.800000,8.500000,NaN,39.000000,NaN,...,70.000000,52.000000,1017.600000,1015.200000,5.000000,5.000000,16.700000,21.100000,NaN,NaN
75%,NaN,NaN,16.800000,28.200000,0.800000,7.400000,10.600000,NaN,48.000000,NaN,...,83.000000,66.000000,1022.400000,1020.000000,7.000000,7.000000,21.600000,26.400000,NaN,NaN


В таблице присутствуют нечисловые столбцы и много NaN-значений.

Сначала приведем нечисловые значения Yes и No в столбцах RainToday и RainTomorrow к 0 и 1. 

In [24]:
weather_df.RainToday.unique()

array(['No', 'Yes'], dtype=object)

In [25]:
weather_df.RainTomorrow.unique()

array(['No', 'Yes'], dtype=object)

In [26]:
weather_df.loc[weather_df.RainToday == "No", "RainToday"] = 0
weather_df.loc[weather_df.RainToday == "Yes", "RainToday"] = 1

weather_df.loc[weather_df.RainTomorrow == "No", "RainTomorrow"] = 0
weather_df.loc[weather_df.RainTomorrow == "Yes", "RainTomorrow"] = 1

In [27]:
weather_df.RainToday.unique()

array([0, 1], dtype=object)

In [28]:
weather_df.RainTomorrow.unique()

array([0, 1], dtype=object)

Разложим дату на три столбца: Year, Mounth, Day.

In [29]:
date_seria = weather_df.Date.str.split(pat = '-')
weather_df['Year'] = date_seria.str[0].astype('int32')
weather_df['Mounth'] = date_seria.str[1].astype('int32')
weather_df['Day'] = date_seria.str[2].astype('int32')

In [30]:
weather_df.drop('Date', axis = 1, inplace = True)

Заполним пропуски в столбце RainToday.

Для этого посмотрим значение в столбце RainTomorrow для вчерашнего дня. 

In [31]:
def fill_value(i, dataset):
    dataset.loc[i, "RainToday"] = dataset['RainTomorrow'].iloc[i - 1]

for i, _ in weather_df[weather_df.RainToday.isnull()].iterrows():
    fill_value(i, weather_df)

In [32]:
weather_df.RainTomorrow = weather_df.RainTomorrow.astype('int8')
weather_df.RainToday = weather_df.RainToday.astype('int8')

Заполним остальные пропуски. 

Там где пропущена максимальная температура, но имеется дневная температура, заполним дневной, а там где пропущена минимальная - утренней.  

То же самое сделаем в обратную сторону для утренней и дневной на основе максимальной и минимальной. 

In [34]:
weather_df.loc[weather_df.MaxTemp.isnull(), "MaxTemp"] = weather_df.Temp3pm[weather_df.MaxTemp.isnull()]
weather_df.loc[weather_df.MinTemp.isnull(), "MinTemp"] = weather_df.Temp9am[weather_df.MinTemp.isnull()]

weather_df.loc[weather_df.Temp3pm.isnull(), "Temp3pm"] = weather_df.MaxTemp[weather_df.MaxTemp.isnull()]
weather_df.loc[weather_df.Temp9am.isnull(), "Temp9am"] = weather_df.MinTemp[weather_df.MinTemp.isnull()]

weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 25 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Location       142193 non-null  object 
 1   MinTemp        141837 non-null  float64
 2   MaxTemp        142013 non-null  float64
 3   Rainfall       140787 non-null  float64
 4   Evaporation    81350 non-null   float64
 5   Sunshine       74377 non-null   float64
 6   WindGustDir    132863 non-null  object 
 7   WindGustSpeed  132923 non-null  float64
 8   WindDir9am     132180 non-null  object 
 9   WindDir3pm     138415 non-null  object 
 10  WindSpeed9am   140845 non-null  float64
 11  WindSpeed3pm   139563 non-null  float64
 12  Humidity9am    140419 non-null  float64
 13  Humidity3pm    138583 non-null  float64
 14  Pressure9am    128179 non-null  float64
 15  Pressure3pm    128212 non-null  float64
 16  Cloud9am       88536 non-null   float64
 17  Cloud3pm       85099 non-null

Остальные пропуски заполним медианой по группе из местоположения, месяца и наличия дождя. 

In [35]:
def to_apply_inputer_creator(means, col):
    def to_apply(x):
        return means.loc[(x['Mounth'], x['Location'], x['RainToday'])][col]
    return to_apply

In [36]:
indexes = ["MinTemp", "MaxTemp", "Temp9am", "Temp3pm", "Pressure3pm", \
    "Pressure9am", "Cloud9am", "Cloud3pm", "Humidity9am", "Humidity3pm", \
    "WindGustSpeed", "WindSpeed9am", "WindSpeed3pm", "Rainfall", "Evaporation", "Sunshine"]

mean_values = weather_df.groupby(["Mounth", "Location", "RainToday"])[indexes].median()
mean_values.head()

MinTemp  MaxTemp  Temp9am  Temp3pm  Pressure3pm  \
Mounth Location RainToday                                                    
1      Adelaide 0             17.1    31.30     22.0    29.50      1012.20   
                1             17.0    23.40     18.7    21.50      1010.40   
       Albany   0             17.0    23.00     20.2    21.50      1014.00   
                1             17.4    20.95     18.7    20.15      1012.10   
       Albury   0             16.7    33.60     23.3    31.75      1010.05   

                           Pressure9am  Cloud9am  Cloud3pm  Humidity9am  \
Mounth Location RainToday                                                 
1      Adelaide 0               1013.8       NaN       NaN         46.0   
                1               1010.5       NaN       NaN         73.0   
       Albany   0               1015.7       6.0       4.0         68.0   
                1               1012.5       8.0       5.5         83.0   
       Albury   0               1013.0       5.5       3.0         50.0   

                           Humidity3pm  WindGustSpeed  WindSpeed9am  \
Mounth Location RainToday                                             
1      Adelaide 0                 28.0           37.0           9.0   
                1                 50.0           41.0          13.0   
       Albany   0                 70.0            NaN          15.0   
                1                 67.0            NaN          17.0   
       Albury   0                 25.0           37.0           9.0   

                           WindSpeed3pm  Rainfall  Evaporation  Sunshine  
Mounth Location RainToday                                                 
1      Adelaide 0                  15.0       0.0          8.8     12.70  
                1                  17.0       6.4          5.0      7.60  
       Albany   0                  23.0       0.0          7.4      9.95  
                1                  24.0       3.0          6.1      6.90  
       Albury   0                  15.0       0.0          NaN       NaN

In [37]:
for idx in tqdm(indexes):
    func = to_apply_inputer_creator(mean_values, idx)
    if weather_df[idx].isnull().sum():
        weather_df.loc[weather_df[idx].isnull(), idx] = weather_df[weather_df[idx].isnull()].apply(func, axis = 1)

100%|██████████| 16/16 [02:05<00:00,  7.86s/it]


In [38]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 25 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Location       142193 non-null  object 
 1   MinTemp        142193 non-null  float64
 2   MaxTemp        142193 non-null  float64
 3   Rainfall       142193 non-null  float64
 4   Evaporation    96711 non-null   float64
 5   Sunshine       90101 non-null   float64
 6   WindGustDir    132863 non-null  object 
 7   WindGustSpeed  136222 non-null  float64
 8   WindDir9am     132180 non-null  object 
 9   WindDir3pm     138415 non-null  object 
 10  WindSpeed9am   142193 non-null  float64
 11  WindSpeed3pm   142193 non-null  float64
 12  Humidity9am    142193 non-null  float64
 13  Humidity3pm    142187 non-null  float64
 14  Pressure9am    130412 non-null  float64
 15  Pressure3pm    130412 non-null  float64
 16  Cloud9am       108159 non-null  float64
 17  Cloud3pm       108152 non-nul

Пока еще не все пропуски заполнены.

Теперь используем группировку по столбцу RainToday. 

In [39]:
indexes = ["Evaporation", "Sunshine", "WindGustSpeed", "Pressure9am", "Pressure3pm", \
           "Cloud9am", "Cloud3pm", "Temp3pm", "Humidity3pm"]
for idx in indexes:
    weather_df.loc[weather_df[idx].isnull() & (weather_df.RainToday == 0), idx] = \
        weather_df[idx][weather_df.RainToday == 0].median()
    weather_df.loc[weather_df[idx].isnull() & (weather_df.RainToday == 1), idx] = \
        weather_df[idx][weather_df.RainToday == 1].median()

Столбцы с нечисловыми значениями заполняем наиболее часто встречающимися значениями.

In [40]:
indexes = ["WindDir9am", "WindGustDir", "WindDir3pm"]
for idx in indexes:
    weather_df.loc[weather_df[idx].isnull() & (weather_df.RainToday == 0), idx] = \
        weather_df[idx][weather_df.RainToday == 0].describe()['top']
    weather_df.loc[weather_df[idx].isnull() & (weather_df.RainToday == 1), idx] = \
        weather_df[idx][weather_df.RainToday == 1].describe()['top']

In [41]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 25 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Location       142193 non-null  object 
 1   MinTemp        142193 non-null  float64
 2   MaxTemp        142193 non-null  float64
 3   Rainfall       142193 non-null  float64
 4   Evaporation    142193 non-null  float64
 5   Sunshine       142193 non-null  float64
 6   WindGustDir    142193 non-null  object 
 7   WindGustSpeed  142193 non-null  float64
 8   WindDir9am     142193 non-null  object 
 9   WindDir3pm     142193 non-null  object 
 10  WindSpeed9am   142193 non-null  float64
 11  WindSpeed3pm   142193 non-null  float64
 12  Humidity9am    142193 non-null  float64
 13  Humidity3pm    142193 non-null  float64
 14  Pressure9am    142193 non-null  float64
 15  Pressure3pm    142193 non-null  float64
 16  Cloud9am       142193 non-null  float64
 17  Cloud3pm       142193 non-nul

Наконец все пропуски заполнены!

Преобразуем нечисловые столбцы Location, WindGustDir, WindDir9am, WindDir3pm к числовому виду с помощью OneHotEncoding. 

In [42]:
print("Уникальных значений для Location:", weather_df.Location.unique().shape[0])
print("Уникальных значений для WindGustDir:", weather_df.WindGustDir.unique().shape[0])
print("Уникальных значений для WindDir9am:", weather_df.WindDir9am.unique().shape[0])
print("Уникальных значений для WindDir3pm:", weather_df.WindDir3pm.unique().shape[0])

Уникальных значений для Location: 49
Уникальных значений для WindGustDir: 16
Уникальных значений для WindDir9am: 16
Уникальных значений для WindDir3pm: 16


In [43]:
weather_df = pd.concat([weather_df, pd.get_dummies(weather_df.Location, prefix='loc')], axis=1)
weather_df.drop("Location", axis = 1, inplace=True)

In [44]:
weather_df = pd.concat([weather_df, pd.get_dummies(weather_df.WindGustDir, prefix='gust')], axis=1)
weather_df.drop("WindGustDir", axis = 1, inplace=True)

In [45]:
weather_df = pd.concat([weather_df, pd.get_dummies(weather_df.WindDir9am, prefix='dir9')], axis=1)
weather_df.drop("WindDir9am", axis = 1, inplace=True)

In [46]:
weather_df = pd.concat([weather_df, pd.get_dummies(weather_df.WindDir3pm, prefix='dir3')], axis=1)
weather_df.drop("WindDir3pm", axis = 1, inplace=True)

Разобьем столбец Mounth на 12 новых столбцов. 

In [47]:
weather_df = pd.concat([weather_df, pd.get_dummies(weather_df.Mounth, prefix='mnth')], axis=1)

In [48]:
print("Новое число столбцов:", weather_df.columns.shape[0])

Новое число столбцов: 130


In [49]:
weather_df.corr().abs()["RainTomorrow"].sort_values(ascending=False)[:20]

RainTomorrow        1.000000
Humidity3pm         0.443087
Sunshine            0.378858
Cloud3pm            0.316670
RainToday           0.310659
Cloud9am            0.263415
Humidity9am         0.257961
Pressure9am         0.238371
Rainfall            0.236689
WindGustSpeed       0.227730
Pressure3pm         0.217022
Temp3pm             0.191811
MaxTemp             0.159382
Evaporation         0.122348
WindSpeed9am        0.090612
WindSpeed3pm        0.085544
MinTemp             0.083328
gust_W              0.062066
loc_Woomera         0.055041
loc_AliceSprings    0.050845
Name: RainTomorrow, dtype: float64

Разделим датасет на тренировочный и обучающий по временному признаку, т.е. на до и после некоторого года. 

In [50]:
print("Минимальное значение года:", weather_df.Year.min())
print("Максимальное значение года:", weather_df.Year.max())

Минимальное значение года: 2007
Максимальное значение года: 2017


Поделим датасет в соотношении 80% обучающих данных и 20% тестовых. 

In [51]:
df_train = weather_df[weather_df.Year <= 2015]
df_test = weather_df[weather_df.Year > 2015]

In [52]:
Y_train = df_train['RainTomorrow']
Y_test = df_test['RainTomorrow']

X_train = df_train.drop('RainTomorrow', axis = 1)
X_test = df_test.drop('RainTomorrow', axis = 1)

In [53]:
print("Количество данных для тестового набора составляет:", str(100 * df_test.shape[0] / weather_df.shape[0]) + "%")

Количество данных для тестового набора составляет: 18.26672199053399%


Сохраним полученные данные.

In [54]:
df_train.to_csv("data/train.csv")
df_test.to_csv("data/test.csv")

Нормализация

In [55]:
scale_features_std = StandardScaler() 

X_train = scale_features_std.fit_transform(X_train) 
X_test = scale_features_std.transform(X_test)